In [30]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Flatten, Reshape
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping

# 훈련에 사용할 이미지들을 지정
image_paths = ['image123.jpg', 'image321.jpg', 'imageee.jpg', 'imageee1.jpg', 'imageeee.jpg']
all_images = []

# 한 이미지를 20번씩 사용함
for path in image_paths:
    img = tf.keras.preprocessing.image.load_img(path, target_size=(200, 200))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    for _ in range(20):
        all_images.append(img_array)

# 이미지 픽셀 값(0~255)을 0~1 사이로 정규화
all_images_array = np.array(all_images) / 255.0

# 오토인코더 모델을 정의
input_img = Input(shape=(200, 200, 3))
x = Flatten()(input_img)
x = Dense(1024, activation='relu')(x)
encoded = Dense(512, activation='relu')(x)

x = Dense(1024, activation='relu')(encoded)
x = Dense(200*200*3, activation='sigmoid')(x)
decoded = Reshape((200, 200, 3))(x)

# 오토인코더
autoencoder = Model(input_img, decoded)

# 4. 모델 컴파일 및 학습

# 학습을 할때 10번 이상 loss 값이 안바뀌면 중간에 학습을 일찍 중지시킴
early_stopping = EarlyStopping(monitor='loss', patience=10, verbose=1)

# 모델 컴파일
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# 모델 학습
autoencoder.fit(all_images_array, all_images_array, epochs=50, batch_size=10, callbacks=[early_stopping])

# 학습된 모델을 저장
tf.keras.models.save_model(autoencoder, 'autoencoder_saved_model2')



Epoch 1/50
10/10 [==============================] - 24s 2s/step - loss: 0.0147
Epoch 2/50
10/10 [==============================] - 27s 3s/step - loss: 0.0088
Epoch 3/50
10/10 [==============================] - 22s 2s/step - loss: 0.0081
Epoch 4/50
10/10 [==============================] - 22s 2s/step - loss: 0.0067
Epoch 5/50
10/10 [==============================] - 22s 2s/step - loss: 0.0073
Epoch 6/50
10/10 [==============================] - 22s 2s/step - loss: 0.0083
Epoch 7/50
10/10 [==============================] - 23s 2s/step - loss: 0.0065
Epoch 8/50
10/10 [==============================] - 22s 2s/step - loss: 0.0058
Epoch 9/50
10/10 [==============================] - 21s 2s/step - loss: 0.0026
Epoch 10/50
10/10 [==============================] - 22s 2s/step - loss: 0.0010
Epoch 11/50
10/10 [==============================] - 21s 2s/step - loss: 5.5254e-04
Epoch 12/50
10/10 [==============================] - 21s 2s/step - loss: 1.7119e-04
Epoch 13/50
10/10 [======================

In [31]:
# 저장된 오토인코더 모델 불러오기
loaded_model = tf.keras.models.load_model('autoencoder_saved_model2')

# 테스트에 사용할 이미지를 로드하고 전처리
img_path = 'error.jpg'
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(200, 200))
img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# 이미지 재구성
reconstructed = loaded_model.predict(img_array)

# 재구성 오차를 계산
mse = np.mean(np.square(img_array - reconstructed))

# 이상 탐지
threshold = 0.01  # 예시임계값, 너무 낮추면 정상을 못찾아냄, 너무 높이면 다 정상이라고 함
if mse > threshold:
    print("이상 데이터입니다.")
else:
    print("정상 데이터입니다.")

1/1 [==============================] - 0s 131ms/step
이상 데이터입니다.


In [46]:
# 테스트에 사용할 이미지를 로드하고 전처리
img_path = 'image09.jpg'
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(200, 200))
img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# 이미지 재구성
reconstructed = loaded_model.predict(img_array)

# 재구성 오차를 계산
mse = np.mean(np.square(img_array - reconstructed))

# 이상 탐지
threshold = 0.02  # 예시임계값, 너무 높이면 다 정상이라고 함
if mse > threshold:
    print("이상 데이터입니다.")
else:
    print("정상 데이터입니다.")

1/1 [==============================] - 0s 99ms/step
이상 데이터입니다.


In [47]:
# 모델 구조 정의
input_img = Input(shape=(200, 200, 3))
x = Flatten()(input_img)
x = Dense(1024, activation='relu')(x)
encoded = Dense(512, activation='relu')(x)
x = Dense(1024, activation='relu')(encoded)
x = Dense(200*200*3, activation='sigmoid')(x)
decoded = Reshape((200, 200, 3))(x)
autoencoder_temp = Model(input_img, decoded)

autoencoder_temp.load_weights('autoencoder_saved_model2/variables/variables')


# 2. "error.jpg" 이미지 불러오기 및 전처리
error_img = tf.keras.preprocessing.image.load_img('imageee.jpg', target_size=(200, 200))
error_img_array = tf.keras.preprocessing.image.img_to_array(error_img) / 255.0
error_img_array = np.expand_dims(error_img_array, axis=0)

# 3. 이미지를 오토인코더에 통과시키기
reconstructed_img_array = autoencoder_temp.predict(error_img_array)

# 4. 원본 이미지와 재구성된 이미지 사이의 재구성 오차 계산
reconstruction_error = np.mean(np.square(error_img_array - reconstructed_img_array))

reconstruction_error

1/1 [==============================] - 0s 120ms/step


2.6746653e-14

In [44]:
# 모델 구조 정의
input_img = Input(shape=(200, 200, 3))
x = Flatten()(input_img)
x = Dense(1024, activation='relu')(x)
encoded = Dense(512, activation='relu')(x)
x = Dense(1024, activation='relu')(encoded)
x = Dense(200*200*3, activation='sigmoid')(x)
decoded = Reshape((200, 200, 3))(x)
autoencoder_temp = Model(input_img, decoded)

autoencoder_temp.load_weights('autoencoder_saved_model2/variables/variables')


# 2. "error.jpg" 이미지 불러오기 및 전처리
error_img = tf.keras.preprocessing.image.load_img('image09.jpg', target_size=(200, 200))
error_img_array = tf.keras.preprocessing.image.img_to_array(error_img) / 255.0
error_img_array = np.expand_dims(error_img_array, axis=0)

# 3. 이미지를 오토인코더에 통과시키기
reconstructed_img_array = autoencoder_temp.predict(error_img_array)

# 4. 원본 이미지와 재구성된 이미지 사이의 재구성 오차 계산
reconstruction_error = np.mean(np.square(error_img_array - reconstructed_img_array))

reconstruction_error

1/1 [==============================] - 0s 133ms/step


0.024797756

In [48]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model

# 1. 데이터 준비
image_paths = ['image123.jpg', 'image321.jpg', 'imageee.jpg', 'imageee1.jpg', 'imageeee.jpg']
all_images = []

for path in image_paths:
    img = tf.keras.preprocessing.image.load_img(path, target_size=(200, 200))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    for _ in range(20):
        all_images.append(img_array)

all_images_array = np.array(all_images) / 255.0

# 2. CNN 오토인코더 모델 정의
input_img = Input(shape=(200, 200, 3))

# Encoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

# 학습을 할때 10번 이상 loss 값이 안바뀌면 중간에 학습을 일찍 중지시킴
early_stopping = EarlyStopping(monitor='loss', patience=10, verbose=1)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# 3. 모델 학습
autoencoder.fit(all_images_array, all_images_array, epochs=50, batch_size=10, callbacks=[early_stopping])

# 4. 학습된 모델 저장
autoencoder.save('autoencoder_cnn_model')


Epoch 1/50
10/10 [==============================] - 11s 1s/step - loss: 0.0218
Epoch 2/50
10/10 [==============================] - 10s 1s/step - loss: 0.0178
Epoch 3/50
10/10 [==============================] - 9s 895ms/step - loss: 0.0128
Epoch 4/50
10/10 [==============================] - 10s 957ms/step - loss: 0.0089
Epoch 5/50
10/10 [==============================] - 10s 998ms/step - loss: 0.0062
Epoch 6/50
10/10 [==============================] - 10s 999ms/step - loss: 0.0045
Epoch 7/50
10/10 [==============================] - 10s 990ms/step - loss: 0.0028
Epoch 8/50
10/10 [==============================] - 9s 921ms/step - loss: 0.0019
Epoch 9/50
10/10 [==============================] - 10s 927ms/step - loss: 0.0016
Epoch 10/50
10/10 [==============================] - 10s 1s/step - loss: 0.0012
Epoch 11/50
10/10 [==============================] - 10s 1s/step - loss: 9.3505e-04
Epoch 12/50
10/10 [==============================] - 10s 1s/step - loss: 8.0636e-04
Epoch 13/50
10/10 [===

In [49]:
# 저장된 오토인코더 모델 불러오기
loaded_model = tf.keras.models.load_model('autoencoder_cnn_model')


In [67]:
# 테스트에 사용할 이미지를 로드하고 전처리
img_path = 'image09.jpg'
img = tf.keras.preprocessing.image.load_img(img_path, target_size=(200, 200))
img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0
img_array = np.expand_dims(img_array, axis=0)

# 이미지 재구성
reconstructed = loaded_model.predict(img_array)

# 재구성 오차를 계산
mse = np.mean(np.square(img_array - reconstructed))

# 이상 탐지
threshold = 0.00019  # 예시임계값, 너무 낮추면 정상을 못찾아냄, 너무 높이면 다 정상이라고 함
if mse > threshold:
    print("이상 데이터입니다.")
else:
    print("정상 데이터입니다.")

1/1 [==============================] - 0s 38ms/step
정상 데이터입니다.


In [61]:
input_img = Input(shape=(200, 200, 3))

# Encoder
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)
autoencoder_temp = Model(input_img, decoded)

autoencoder_temp.load_weights('autoencoder_cnn_model/variables/variables')


# 2. "error.jpg" 이미지 불러오기 및 전처리
error_img = tf.keras.preprocessing.image.load_img('image09.jpg', target_size=(200, 200))
error_img_array = tf.keras.preprocessing.image.img_to_array(error_img) / 255.0
error_img_array = np.expand_dims(error_img_array, axis=0)

# 3. 이미지를 오토인코더에 통과시키기
reconstructed_img_array = autoencoder_temp.predict(error_img_array)

# 4. 원본 이미지와 재구성된 이미지 사이의 재구성 오차 계산
reconstruction_error = np.mean(np.square(error_img_array - reconstructed_img_array))

reconstruction_error

1/1 [==============================] - 0s 121ms/step


0.00018554466